# Lab2: Memory Process Analysis

Identifying memory archive with suspicious process. Notebook run on a [WIN-FOR](https://github.com/digitalsleuth/WIN-FOR) configured machine.

In [1]:
import os

print(f"Jupyter Lab running in directory {os.getcwd()}")

Jupyter Lab running in directory C:\Users\vboxuser\repos\CYBS-5014\lab2


In [2]:
# Setup

from pathlib import Path
import subprocess, hashlib, json, textwrap, os, sys, shlex
IMG = Path(r"..\iveep\CTICOE-iveep_374_2025-03-17T17-27Z.mem")
CASE_DIR  = Path.cwd() / "output"
VOL = Path(r"C:\Users\vboxuser\repos\volatility3\vol.py")
VOL2 = Path(r"C:\Users\vboxuser\repos\volatility\vol.py")
if not IMG.is_file():
    raise FileNotFoundError("File does not exist")

print(f"Analsysis ready for {IMG}")

Analsysis ready for ..\iveep\CTICOE-iveep_374_2025-03-17T17-27Z.mem


# Hashing the Memory file

Validate that the file is unchanged before and after analysis.

In [3]:
def file_hashes(p: Path, chunksize=2**20):
    h = { 'md5': hashlib.md5(), 'sha256': hashlib.sha256() }
    with p.open('rb') as f:
        while chunk := f.read(chunksize):
            h['md5'].update(chunk); h['sha256'].update(chunk)
    return {k:v.hexdigest() for k,v in h.items()}

hashes = file_hashes(IMG)
(h := (CASE_DIR  / "evidence_hashes.json")).write_text(json.dumps({'image': IMG.name, **hashes}, indent=2))
hashes

{'md5': '869766b530db76e2ebb5a76977c69871',
 'sha256': 'ae86d42971ea187a60e0a4f495509714c1af93656134914f6d42bd92c1df5c30'}

# Volatility Wrapper Functions

In [4]:
def vol(plugin, *args):
    cmd = [sys.executable, str(VOL), "-f", str(IMG), str(plugin)] + [str(a) for a in args]
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    if p.returncode != 0:
        sys.stderr.write(err)
    return out

def vol2(plugin, *args):
    cmd = [sys.executable, str(VOL2), "-f", str(IMG), str(plugin)] + [str(a) for a in args]
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    if p.returncode != 0:
        sys.stderr.write(err)
    return out

def save(txt, name):
    out_path = os.path.join(str(CASE_DIR), str(name))
    if not os.path.isdir(str(CASE_DIR)):
        try:
            os.makedirs(str(CASE_DIR))
        except OSError:
            pass  # already exists
    with open(out_path, 'wb') as f:
        f.write(txt)
    return out_path

from __future__ import print_function

def printvol(txt):
    return print(txt.decode('utf-8'))

# Determine OS/banner, KDBG, and Basic Environment

Running volatility's info/banner plugin to identify:
- OS Family/Version
- Kernel Debugger Block (KDBG)
- Symbol Context

In [5]:
info = vol("windows.info")
save(info, "01_windows.info.txt")
printvol(info)

Volatility 3 Framework 2.26.2

Variable	Value

Kernel Base	0xf80419400000
DTB	0x1ad000
Symbols	file:///C:/Users/vboxuser/repos/volatility3/volatility3/symbols/windows/ntkrnlmp.pdb/A970BE3E9F5BB6D521F0CA2A4B5B51AC-1.json.xz
Is64Bit	True
IsPAE	False
layer_name	0 WindowsIntel32e
memory_layer	1 Elf64Layer
base_layer	2 FileLayer
KdVersionBlock	0xf8041a00f410
Major/Minor	15.19041
MachineType	34404
KeNumberProcessors	4
SystemTime	2025-03-17 17:27:09+00:00
NtSystemRoot	C:\Windows
NtProductType	NtProductWinNt
NtMajorVersion	10
NtMinorVersion	0
PE MajorOperatingSystemVersion	10
PE MinorOperatingSystemVersion	0
PE Machine	34404
PE TimeDateStamp	Sun May 29 09:00:14 2067



Observations:
- Windows 10 x64 kernel: NtMajorVersion 10, NtMinorVersion 0, Is64Bit True; multi-processor kernel (ntkrnlmp) with 4 logical CPUs (KeNumberProcessors 4).
- Kernel build line: Major/Minor 15.19041 indicates the 19041 kernel family.
- Architecture identifiers: MachineType 34404 / PE Machine 34404 -> AMD64.
- System time reference: SystemTime 2025-03-17 17:27:09+00:00 (UTC), will be used as the snapshot anchor for any timeline correlation.

# Process Enumeration

In [6]:
info = vol("windows.pslist")
print("pslist output")
printvol(info)
pslist = save(info, "02_pslist.txt")

pslist output
Volatility 3 Framework 2.26.2

PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	File output

4	0	System	0x840e90261040	213	-	N/A	False	2025-03-12 16:39:40.000000 UTC	N/A	Disabled
108	4	Registry	0x840e9037c040	4	-	N/A	False	2025-03-12 16:39:25.000000 UTC	N/A	Disabled
536	4	smss.exe	0x840e910b4040	2	-	N/A	False	2025-03-12 16:39:40.000000 UTC	N/A	Disabled
804	796	csrss.exe	0x840e921650c0	13	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
880	796	wininit.exe	0x840e9228c080	1	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
888	872	csrss.exe	0x840e92291140	10	-	1	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
980	872	winlogon.exe	0x840e92305080	3	-	1	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
1000	880	services.exe	0x840e9230b100	9	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
548	880	lsass.exe	0x840e92349080	10	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
756	1000	svchost.exe	0x840e923de2c0	17	-	0	False	

In [7]:
info = vol("windows.psscan")
print("psscan output")
printvol(info)
psscan = save(info, "03_psscan.txt")

psscan output
Volatility 3 Framework 2.26.2

PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	File output

4	0	System	0x840e90261040	213	-	N/A	False	2025-03-12 16:39:40.000000 UTC	N/A	Disabled
108	4	Registry	0x840e9037c040	4	-	N/A	False	2025-03-12 16:39:25.000000 UTC	N/A	Disabled
7152	1000	svchost.exe	0x840e90ad3080	4	-	1	False	2025-03-12 16:41:30.000000 UTC	N/A	Disabled
9452	1000	svchost.exe	0x840e90b0d080	5	-	2	False	2025-03-12 19:20:09.000000 UTC	N/A	Disabled
3604	1000	svchost.exe	0x840e90c19340	7	-	1	False	2025-03-12 16:41:35.000000 UTC	N/A	Disabled
1480	1000	svchost.exe	0x840e90de2080	10	-	0	False	2025-03-12 16:41:34.000000 UTC	N/A	Disabled
8592	756	FileCoAuth.exe	0x840e90de9080	2	-	1	False	2025-03-12 16:43:45.000000 UTC	N/A	Disabled
536	4	smss.exe	0x840e910b4040	2	-	N/A	False	2025-03-12 16:39:40.000000 UTC	N/A	Disabled
14128	1000	svchost.exe	0x840e918e90c0	2	-	0	False	2025-03-17 17:16:17.000000 UTC	N/A	Disabled
14536	15172	spoolsv.exe	0x840e91a

In [8]:
info = vol("windows.pstree")
print("pstree output")
printvol(info)
pstree = save(info, "04_pstree.txt")

pstree output
Volatility 3 Framework 2.26.2

PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	Audit	Cmd	Path

4	0	System	0x840e90261040	213	-	N/A	False	2025-03-12 16:39:40.000000 UTC	N/A	-	-	-
* 536	4	smss.exe	0x840e910b4040	2	-	N/A	False	2025-03-12 16:39:40.000000 UTC	N/A	\Device\HarddiskVolume3\Windows\System32\smss.exe	\SystemRoot\System32\smss.exe	\SystemRoot\System32\smss.exe
* 1924	4	MemCompression	0x840e966b1080	70	-	N/A	False	2025-03-12 16:39:53.000000 UTC	N/A	MemCompression	-	-
* 108	4	Registry	0x840e9037c040	4	-	N/A	False	2025-03-12 16:39:25.000000 UTC	N/A	Registry	-	-
804	796	csrss.exe	0x840e921650c0	13	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	\Device\HarddiskVolume3\Windows\System32\csrss.exe	%SystemRoot%\system32\csrss.exe ObjectDirectory=\Windows SharedSection=1024,20480,768 Windows=On SubSystemType=Windows ServerDll=basesrv,1 ServerDll=winsrv:UserServerDllInitialization,3 ServerDll=sxssrv,4 ProfileControl=Off MaxRequestThreads=16	C

In [9]:
info = vol("windows.psxview")
print("psxview output")
printvol(info)
pstree = save(info, "05_psxview.txt")

psxview output
Volatility 3 Framework 2.26.2

Offset(Virtual)	Name	PID	pslist	psscan	thrdscan	csrss	Exit Time

0x840e92fc7080	svchost.exe	1988	True	True	True	True	
0x840e96d9f080	svchost.exe	3308	True	True	True	True	
0x840e9783b080	CSFalconContai	6604	True	True	True	True	
0x840e970f3080	svchost.exe	3172	True	True	True	True	
0x840e96e77080	svchost.exe	3636	True	True	True	True	
0x840e974b7080	CSFalconContai	5980	True	True	True	True	
0x840e96e0f080	sihost.exe	6908	True	True	True	True	
0x840e90ad3080	svchost.exe	7152	True	True	True	True	
0x840e99387080	StartMenuExper	7408	True	True	True	True	
0x840e98aef080	RuntimeBroker.	7684	True	True	True	True	
0x840e91e1e080	userinit.exe	9940	True	True	False	False	2025-03-12 19:20:36+00:00
0x840e9e99e080	dllhost.exe	7884	True	True	True	True	
0x840e96c23280	spoolsv.exe	2904	True	True	True	True	
0x840e976a2300	unsecapp.exe	5220	True	True	True	True	
0x840e99760080	msedge.exe	7484	True	True	False	False	2025-03-12 19:21:14+00:00
0x840e9c441080	SearchApp.exe

Observations:
- Computer start time: ~ 2025-03-12 16:39:20+00:00 (UTC). Compare that to SystemTime 2025-03-17 17:27:09+00:00 (UTC), the machine has been active for roughly 5 days.
- In psxview, no applications are obviously hiding from process listings.

Suspicious candidates:

1) excel-update.exe (PID 4672)

- PSEXESCV.exe (PID 11296) was spawned by services.exe (PID 1000). Is is a service dropped/used by Sysinternals PsExec for remote execution. Its presence on an endpoint is a lateral-movement indicator unless you can account for legitimate admin use. While it's inclusion is not a malicious indicator alone, the fact that it spawns an process of an executable named "excel-update.exe" is highly suspicious.
- excel-update.exe (PID 4672) was spawned by PSEXESVC.exe (11296), which is a highly suspicious executable. 'excel-update.exe' is not a standard file name for Microsoft Excel updates, which are typically handled by the OfficeC2RClient.exe or downloaded as standalone .exe files with specific Knowledge Base (KB) numbers. Additionally, this binary is in the Windows root and launched by a remote-execution service also in the root folder.
- cmd.exe (PID 13752) and powershell.exe (PID 13156) are both suspicious because it was immediately spawned by this excel-update.exe. This is a typical example of a living-of-the-land binary (LOLBin) attack.

2) spoolsv.exe (PID 14536)

- This spoolsv.exe (PID 14536) is the child of a different powershell.exe (PID 15172). The legitimate print spooler service, spoolsv.exe (PID 2904), is running correctly as a child of services.exe (PID 1000). This second spoolsv.exe is launched by the user IanVeep (explorer.exe, PID 9852, Session 2). A user's PowerShell should never spawn spoolsv.exe. It is additionally spawned between the time of the above cmd.exe (PID 13752) spawn time and powershell.exe (PID 13156) spawn time, which adds onto the suspicion. This is a clear-cut case of an attacker naming their malicious payload spoolsv.exe to blend in with normal system processes.

3) ScreenConnect.ClientService.exe (PID 6988)

- ScreenConnect (now ConnectWise Control) is a legitimate Remote Access Tool (RAT), similar to TeamViewer. It's also frequently abused by attackers for persistent remote access. The pstree command line shows it connecting to instance-gmpdbd-relay.screenconnect.com. This could be a legitimate IT support session, but given the other malicious activity, it is highly suspect and should be verified as an authorized connection.

The entire chain above in part 1, and spawning of the process in part 2 and 3, takes place just minutes before the documented System Time. While the context is missing for this memory image, it's possible the end-user or tool noticed suspicious actions occurring and an image was taken soon after. This further confirms these as supicious processes.

# Process network communication

In [10]:
info = vol("windows.netscan.NetScan")
print("windows netscan output")
printvol(info)
pstree = save(info, "06_netscan.txt")

windows netscan output
Volatility 3 Framework 2.26.2

Offset	Proto	LocalAddr	LocalPort	ForeignAddr	ForeignPort	State	PID	Owner	Created

0x840e9028e470	TCPv4	0.0.0.0	49668	0.0.0.0	0	LISTENING	2904	spoolsv.exe	2025-03-12 16:40:03.000000 UTC
0x840e9028e890	TCPv4	0.0.0.0	445	0.0.0.0	0	LISTENING	4	System	2025-03-12 16:40:05.000000 UTC
0x840e9028e890	TCPv6	::	445	::	0	LISTENING	4	System	2025-03-12 16:40:05.000000 UTC
0x840e90aa1b40	TCPv4	10.20.20.23	59848	20.10.127.192	443	ESTABLISHED	10224	MsSense.exe	2025-03-17 17:26:17.000000 UTC
0x840e90e64010	TCPv4	10.20.20.23	50167	23.194.127.11	443	CLOSE_WAIT	4236	SearchApp.exe	2025-03-12 16:51:49.000000 UTC
0x840e90ed0500	UDPv4	10.20.20.23	138	*	0		4	System	2025-03-12 16:39:44.000000 UTC
0x840e90ed0820	UDPv4	10.20.20.23	137	*	0		4	System	2025-03-12 16:39:44.000000 UTC
0x840e911dd470	TCPv4	10.20.20.23	139	0.0.0.0	0	LISTENING	4	System	2025-03-12 16:39:44.000000 UTC
0x840e911dd730	TCPv4	0.0.0.0	135	0.0.0.0	0	LISTENING	1076	svchost.exe	2025-03-12 16:39:5

Observations:

1) excel-update.exe (PID 4672):
- ESTABLISHED 10.20.20.23:59231 → 35.86.58.69:4443 (PID 4672 excel-update.e) at 2025-03-17 17:01:19 UTC.
- Immediately after PSEXESVC.exe spawns it, this unknown binary makes an outbound session to a nonstandard TLS port (4443) on 35.86.58.69. That same IP is later used by the rogue spoolsv.exe (below), tying the chain together.

2) spoolsv.exe (PID 14536)
- CLOSED 10.20.20.23:59329 → 35.86.58.69:4444 at 17:12:10 (same minute this spoolsv starts).
- Multiple HTTP sessions to the same host:
  - CLOSED 10.20.20.23:59867/59865/59883/59882/59881 → 35.86.58.69:80 around 17:26:37–17:27:06.
  - ESTABLISHED 10.20.20.23:59884 → 35.86.58.69:80 at 17:27:09.
- Port 4444 and repeated hits to the same external IP are red flags for C2/handshake behavior.
- Legitimate print spooler traffic doesn’t originate as a child of PowerShell in a user session, and it typically doesn’t initiate outbound HTTP/4444 to the internet.
- The legit print spooler (spoolsv.exe PID 2904) is also present and only listens on RPC/ephemeral (LISTEN 0.0.0.0:49668), which is normal. Distinguish it from the rogue one by PID/parent/session and its outbound connections.

3) ScreenConnect.* — interactive remote access active
- ESTABLISHED 10.20.20.23:59262 → 139.178.70.144:443 (PID 6988 ScreenConnect.) at 17:05:19.
- Confirms active remote-access tooling shortly after the PsExec/excel-update activity. While potentially legitimate, in the context of the other confirmed malware, this connection is highly suspect and could represent unauthorized remote access by the attacker.

# Summary

Memory analysis indicates a high-confidence system compromise occurring minutes before the snapshot. Evidence points to lateral movement using PsExec (PSEXESVC.exe, PID 11296) originating from services.exe, followed by the execution of suspicious, masquerading payloads. Key activities include command-and-control (C2) communication and a potentially unauthorized remote access session via ScreenConnect, all clustered within approximately 25 minutes prior to the memory capture. System OS is Windows 10 x64 (Build 19041), uptime ~5 days, with no obvious process hiding detected.

The primary malicious activity chain involved PSEXESVC.exe launching excel-update.exe (PID 4672) from C:\Windows, which immediately established C2 traffic to 35.86.58.69:4443 and spawned cmd.exe (PID 13752) and powershell.exe (PID 13156). Concurrently, a separate user-initiated powershell.exe (PID 15172, Session 2) launched a second, rogue spoolsv.exe (PID 14536), distinct from the legitimate system process (PID 2904). This rogue spoolsv.exe also initiated C2 connections to the same IP (35.86.58.69) on ports 4444 and 80, confirming its link to the intrusion.

Additionally, an active ScreenConnect remote access session (ScreenConnect.ClientService.exe, PID 6988) established outbound communication to 139.178.70.144:443 around the same timeframe. While potentially legitimate, its timing relative to the confirmed malware execution makes it highly suspect. Immediate actions should include isolating the host (10.20.20.23), blocking the C2 IP (35.86.58.69), verifying the ScreenConnect session's legitimacy, and initiating further forensic investigation.

---

# Process network communication (VOLATILITY 2)

Netscan plugin is not supported in the image being analyzed (Windows XP), so the connscan and sockets will be used for analysis instead. 

This requires Volatility2, so change to an appropriate environment for this execution and re-intialize setup and helper Jupyter cells.

In [11]:
info = vol2("connscan")
print("connscan")
printvol(info)
pstree = save(info, "07_connscan.txt")

TypeError: write() argument must be str, not <class 'bytes'>

In [ ]:
info = vol2("sockets")
print("sockets")
printvol(info)
pstree = save(info, "08_sockets.txt")